In [21]:
import pandas as pd
import math
from string import ascii_lowercase, ascii_uppercase
import re
import pepfeature as pf

In [22]:
df = pd.read_csv('peptides-complete.csv')

/tmp/ipykernel_9989/3983260913.py:1: DtypeWarning: Columns (16,17,23,24,31,32,38,53,54,57,63,64,65,66,67,68,71,76,79) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('peptides-complete.csv')


In [23]:
# Drop the unnecessary columns and set the dtypes
used_columns = {'ID':int, 
                'COMPLEXITY':str,
                'SEQUENCE':str,
                'HEMOLITIC CYTOTOXIC ACTIVITY - TARGET CELL':str,
                'HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS GROUP':str,
                'HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS VALUE':str,
                'HEMOLITIC CYTOTOXIC ACTIVITY - CONCENTRATION':str,
                'HEMOLITIC CYTOTOXIC ACTIVITY - UNIT':str,
                'HEMOLITIC CYTOTOXIC ACTIVITY - REFERENCE':object}
df = df[used_columns.keys()]
df = df.astype(used_columns)
df['HEMOLITIC CYTOTOXIC ACTIVITY - REFERENCE'] = df['HEMOLITIC CYTOTOXIC ACTIVITY - REFERENCE'].fillna(1)   # Assign 1 to the missing references.

In [24]:
# min_len = 10
# max_len = 50

# df['len'] = df['SEQUENCE'].str.len()
# df = df[df['len'] <= max_len]
# df = df[df['len'] >= min_len]
# df = df[df['COMPLEXITY'] == 'Monomer']
# df = df[df['HEMOLITIC CYTOTOXIC ACTIVITY - TARGET CELL'] != 'nan']
# x = df.groupby(['HEMOLITIC CYTOTOXIC ACTIVITY - TARGET CELL']).size().reset_index(name='counts')
# x = x.sort_values(by=['counts'], ascending=False)
# with pd.option_context('display.max_rows', None,):
#    print(x)


In [25]:
# Drop the unwanted rows, including target cells other than human erythrocytes, lysis groups other than numeric and multimers.
df = df[df['HEMOLITIC CYTOTOXIC ACTIVITY - TARGET CELL'] == 'Human erythrocytes']
df = df[df['HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS GROUP'] != 'nan']
df = df[df['HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS GROUP'] != 'MIC']
df = df[df['HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS GROUP'] != 'MHC']
df = df[df['COMPLEXITY'] == 'Monomer']

In [26]:
# Group the dataframe by peptide and keep the data from the most frequent reference.
grouped = df.groupby(['SEQUENCE'])

def keep_most_frequent(group):
    counts = group['HEMOLITIC CYTOTOXIC ACTIVITY - REFERENCE'].value_counts()
    most_frequent = counts.idxmax()
    return group[group['HEMOLITIC CYTOTOXIC ACTIVITY - REFERENCE'] == most_frequent]

def keep_largest_concentration(group):
    return group[group['HEMOLITIC CYTOTOXIC ACTIVITY - CONCENTRATION'] == group['HEMOLITIC CYTOTOXIC ACTIVITY - CONCENTRATION'].max()].head(1)

# grouped = grouped.apply(keep_most_frequent)
grouped = grouped.apply(keep_largest_concentration)
grouped = grouped.aggregate(lambda x: x.to_list())
grouped.to_csv('filtered.csv', index=False)

In [27]:
# Drop unnecessary columns and rename the columns.
grouped = grouped[['ID', 'SEQUENCE',
                   'HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS GROUP',
                   'HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS VALUE',
                   'HEMOLITIC CYTOTOXIC ACTIVITY - CONCENTRATION',
                   'HEMOLITIC CYTOTOXIC ACTIVITY - UNIT',]]
df = grouped.rename(columns={'ID':'id',
                             'SEQUENCE':'sequence',
                             'HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS GROUP':'lysis_group',
                             'HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS VALUE':'lysis_value',
                             'HEMOLITIC CYTOTOXIC ACTIVITY - CONCENTRATION':'concentration',
                             'HEMOLITIC CYTOTOXIC ACTIVITY - UNIT':'unit'})

In [28]:
# Filter out the peptide with lower case letters in it.
df['sequence'] = df['sequence'].str.replace(' ', '')
for lower_case in ascii_lowercase:
    df = df[df['sequence'].str.contains(lower_case) == False]

In [29]:
# Filter out the peptide with unnatural amino acids in it.
unnatural_aa = 'BJOUXZ'
for aa in unnatural_aa:
    df = df[df['sequence'].str.contains(aa) == False]

In [30]:
# Drop the peptides with sequence length less than 10 or greater than 50.
min_len = 10
max_len = 50

df['len'] = df['sequence'].str.len()
df = df[df['len'] <= max_len]
df = df[df['len'] >= min_len]
df = df.drop(columns=['len'])
df = df[df['sequence'].notna()]

In [31]:
df

,,id,sequence,lysis_group,lysis_value,concentration,unit
SEQUENCE,,,,,,,
AAAAAAAAAAGIGKFLHSAKKFGKAFVGEIMNS,45932,5613,AAAAAAAAAAGIGKFLHSAKKFGKAFVGEIMNS,0-10% Hemolysis,9% Hemolysis,200,µg/ml
AAAAAAAAAK,90806,11330,AAAAAAAAAK,50-60% Hemolysis,55% Hemolysis,500,µg/ml
AAAKAALNAVLVGANA,4288,618,AAAKAALNAVLVGANA,50-60% Hemolysis,50% Hemolysis,40,µM
AAGKGLVSNLLEK,95865,12109,AAGKGLVSNLLEK,0-10% Hemolysis,0% Hemolysis,80,µM
AAGMGFFGAR,11503,1217,AAGMGFFGAR,0-10% Hemolysis,0% Hemolysis,3.12-100,µM
...,...,...,...,...,...,...,...
YYHFWHRGVTKR,96810,12207,YYHFWHRGVTKR,0-10% Hemolysis,<5% Hemolysis,150,µM
YYHFWHRGVTKRSLSPHRPR,96780,12203,YYHFWHRGVTKRSLSPHRPR,0-10% Hemolysis,<5% Hemolysis,150,µM
YYHFWHRGVTKRSLSPHRPRHSR,96770,12202,YYHFWHRGVTKRSLSPHRPRHSR,0-10% Hemolysis,<5% Hemolysis,150,µM


In [32]:
# Calculate the molecular weight of the peptides.
pf.aa_molecular_weight.calc_df(df, Ncores=1, aa_column='sequence')
df['feat_molecular_weight'] = df['feat_molecular_weight'].astype(float)

In [33]:
# Parse the lysis value from string to float.
df['lysis_value'] = df['lysis_value'].str.lstrip('<>')
df['lysis_value'] = df['lysis_value'].str.split('%').str[0]

def float_ignore_plus_minus(x):
    x = re.sub(r'[A-Za-z]', '', x)    # Remove letters
    try:
        if '-' in x:
            return float(x.split('-')[1])    # Return the larger lysis value if the original data is a range.
        else:
            return sum(map(float,x.replace('l', '1').strip('()=≤').split("±"))) # Return the maximum possible lysis value if the original data is a range.
    except:
        return float("NaN")                         # Return NaN if the data cannot be parsed.
    
df['lysis_value'] = df['lysis_value'].apply(float_ignore_plus_minus)
df['lysis_value'] = df['lysis_value'].astype(float)

In [34]:
# Parse the concentration from string to float.
def conc_str_process(concentration_str: str):
    concentration_str = concentration_str.replace("–","-")
    concentration_str = concentration_str.replace("->","-")
    concentration_str = concentration_str.replace(",",".") 

    if concentration_str[0] == '<' or concentration_str[0] == '>':
        concentration = float_ignore_plus_minus(concentration_str.strip('<=> '))
    elif '-' in concentration_str:
        concentrations = concentration_str.split("-")
        concentration = float_ignore_plus_minus(concentrations[0]) + float_ignore_plus_minus(concentrations[1])
        concentration /= 2
    else:
        concentration = float_ignore_plus_minus(concentration_str)
    return concentration

df['concentration'] = df['concentration'].apply(lambda x: conc_str_process(x))
df = df[df['concentration'].notna()]
df['concentration'] = df['concentration'].astype(float)

/tmp/ipykernel_9989/122250370.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['concentration'] = df['concentration'].astype(float)


In [35]:
# Unify the unit of peptide concentrations to µg/ml.

df = df[df['unit'].notna()]
mask = (df['unit'] == 'µM')
df.loc[mask,'concentration'] = df.loc[mask,'concentration'] * df.loc[mask,'feat_molecular_weight'] / 1000
df = df[df['concentration'] <= 300]
df['concentration'] = df['concentration'].astype(float)
df['concentration_uM'] = df['concentration'] * 1000 / df['feat_molecular_weight']

In [36]:
# Keep only the columns we need and reset the index.
df = df[['id', 'sequence', 'lysis_group', 'lysis_value', 'concentration', 'concentration_uM']]
df = df.reset_index(drop=True)

In [37]:
# Set proper data types for columns.
df['lysis_group'] = df['lysis_group'].astype('category')
df['sequence'] = df['sequence'].astype('string')

In [38]:
df

,id,sequence,lysis_group,lysis_value,concentration,concentration_uM
0,5613,AAAAAAAAAAGIGKFLHSAKKFGKAFVGEIMNS,0-10% Hemolysis,9.0,200.000000,53.274376
1,618,AAAKAALNAVLVGANA,50-60% Hemolysis,50.0,67.794400,40.000000
2,12109,AAGKGLVSNLLEK,0-10% Hemolysis,0.0,121.256000,80.000000
3,1217,AAGMGFFGAR,0-10% Hemolysis,0.0,59.101681,51.560000
4,13172,AAKIILNPKFR,10-20% Hemolysis,10.0,185.690880,128.000000
...,...,...,...,...,...,...
2979,16558,YNYSYIITRS,0-10% Hemolysis,2.0,72.076500,50.000000
2980,12173,YPCKLNLKLGKVPFH,0-10% Hemolysis,0.0,81.681297,40.650000
2981,5670,YSKSLPLSVLNP,0-10% Hemolysis,5.0,256.000000,168.899973
2982,12207,YYHFWHRGVTKR,0-10% Hemolysis,5.0,277.206000,150.000000


In [39]:
df.to_csv('processed.csv', index=False)

In [40]:
df.shape

(2984, 6)